# TCP编程

Socket是网络编程的一个抽象概念，通常我们用一个Socket表示打开了一个网络连接，而打开一个Socket需要知道目标计算机的IP地址和端口号，再指定协议类型即可

## 客户端

大多数连接都是可靠的TCP连接，创建TCP连接时，主动发起的叫客户端，被动响应连接的叫服务器。

当我们在浏览器中访问新浪时，我们自己的计算机就是客户端，浏览器会主动向新浪的服务器发起连接，如果一切顺利，新浪的服务器接受了我们的连接，一个TCP连接就建立起来，后面的通信就是发送网页内容了。

### 创建一个基于TCP连接的Socket 

In [1]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('www.sina.com.cn', 80))

- socket.AF_INET指定使用IPv4协议，如果要使用更加先进的IPV6，可以指定为AF_INET6
- socket.SOCK_STREAM指定使用面向流的TCP协议
- 客户端要主动发起TCP连接，必须知道服务器的IP地址和端口号。新浪网站的IP地址可以用域名www.sina.com.cn自动转换到IP地址
- 端口号，作为服务器提供什么样的服务，端口号就必须固定下来。由于我们想访问网页，因此新浪提供网页服务的服务器，必须把端口固定在80端口，因为80端口是web服务器的标准端口。其他服务都有对应的标准端口号，例如SMTP服务是25端口，FTP服务是21端口，端口号小于1024的是Internet标准服务的端口，端口号大于1024的，可以任意使用。
- 地址参数是一个tuple

### 发送请求，要求返回首页的内容

In [2]:
s.send(b'GET / HTTP/1.1\r\nHost: www.sina.com.cn\r\nConnection: close\r\n\r\n')
buffer = []
while True:
    # 每次最多接收1K字节：
    d = s.recv(1024)
    if d:
        buffer.append(d)
    else:
        break
data = b''.join(buffer)
s.close()

### 保存文件

In [3]:
header, html = data.split(b'\r\n\r\n', 1)
print(header.decode('utf-8'))
with open('sina.html', 'wb') as f:
    f.write(html)

HTTP/1.1 200 OK
Server: nginx
Date: Sun, 15 Apr 2018 15:39:01 GMT
Content-Type: text/html
Content-Length: 603685
Connection: close
Last-Modified: Sun, 15 Apr 2018 15:37:08 GMT
Vary: Accept-Encoding
Expires: Sun, 15 Apr 2018 15:40:00 GMT
Cache-Control: max-age=60
X-Powered-By: shci_v1.03
Age: 1
Via: http/1.1 cnc.beixian.ha2ts4.205 (ApacheTrafficServer/6.2.1 [cMsSfW]), http/1.1 cmcc.nanjing.ha2ts4.239 (ApacheTrafficServer/6.2.1 [cHs f ])
X-Via-Edge: 1523806741319f651d3b7e53519703bd258ae
X-Cache: MISS.MERGE.239
X-Via-CDN: f=edge,s=cmcc.nanjing.ha2ts4.238.nb.sinaedge.com,c=183.211.81.246;f=Edge,s=cmcc.nanjing.ha2ts4.239,c=112.25.53.238


-  接收数据时，调用recv(max)方法，一次最多接收指定的字节数，因此在一个while循环中反复接收，直到recv()返回空数据，表示接收完毕，退出循环。
- 接收完数据后，调用close()方法关闭Socket，这样一次完整的网络通信就结束了
- 接收到的数据包含HTTP头和网页本身，我们需要使用split分离HTTP头和网页

## 服务器

- 服务器进程首先要绑定一个端口并监听来自其他客户端的连接，如果某个客户端连接过来了，服务器就与该客户端建立socket连 接，随后的通信就靠这个Socket连接了。

- 一个socket依赖4项：服务器地址，服务器端口，客户端地址，客户端端口来唯一确定一个Socket

- 服务器需要同时响应多个客户端的请求，所以每个连接都需要一个新的进程或者新的线程来处理，否则，服务器一次就只能服务一个客户端了。

编写一个简单的服务器程序，它接受客户端的连接，把客户端发来的字符串加上hello再发回去

### 创建一个基于IPv4和TCP协议的Socket： 

In [4]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

### 绑定监听的地址和端口

- 服务器可能有多块网卡，可以绑定到某一块网卡的IP地址上，也可以用0.0.0.0绑定到所有的网络地址上，还可以用127.0.0.1绑定到本机地址。
- 端口号需要预先指定，由于我们的这个服务不是标准服务，所以用9999这个端口号，小于1024的端口号必须要有管理员权限才能绑定

In [5]:
s.bind(('127.0.0.1', 9999))

### 调用listen()方法开始监听端口，传入的参数指定等待连接的最大数量

In [6]:
s.listen(5)
print('Waiting for connection...')

Waiting for connection...


### 服务器通过一个永久循环来接受来自客户端的请求，每个连接需要创建新线程来处理，否则， 单线程在处理连接的过程中，无法接受其他客户端的连接：

In [ ]:
import threading
import time
def tcplink(sock, addr):
    print("Accept new connection from %s:%s..." % addr)
    sock.send(b'Welcome!')
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if not data or data.decode('utf-8') == 'exit':
            break
        sock.send(('Hello, %s!' % data.decode('utf-8')).encode('utf-8'))
    sock.close()
    print('Connection from %s:%s closed.'% addr)

In [ ]:
while True:
    sock, addr = s.accept()
    t = threading.Thread(target=tcplink, args=(sock, addr))
    t.start()


Accept new connection from 127.0.0.1:51344...
Connection from 127.0.0.1:51344 closed.


### 客户端程序:

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 建立连接:
s.connect(('127.0.0.1', 9999))
# 接收欢迎消息:
print(s.recv(1024).decode('utf-8'))
for data in [b'Michael', b'Tracy', b'Sarah']:
    # 发送数据:
    s.send(data)
    print(s.recv(1024).decode('utf-8'))
s.send(b'exit')
s.close()